In [37]:
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime as dt 
gas = pd.read_csv('./PredictedGasPrices ')
gas.set_index('Dates', inplace= True)
gas.drop(columns= 'Unnamed: 0')
gas

,Unnamed: 0,LSTM Pred
Dates,,
2024-10-31,0,12.014008
2024-11-30,1,12.207607
2024-12-31,2,12.432805
2025-01-31,3,12.550012
2025-02-28,4,12.555716
2025-03-31,5,12.497707
2025-04-30,6,12.421888
2025-05-31,7,12.223061
2025-06-30,8,12.024046


## Gas Contract Pricing Function
This fuction can be used to price the gas contract assuming the price is in the data predicted in the previous task. If daily or weekly prices are required it can be done by changing the base model, however for simplicity's sake I will use the monthy predicted data from the model.

In [143]:

def dater(x): #makes handling dates faster
    x = dt.strptime(x,"%Y-%m-%d").date()
    return x
#self.index.get_loc
def price_gas_contract(injection_dates, withdrawal_dates, injection_rates, withdrawal_rates, max_volume, storage_costs):
#starting variables
    current_stored = 0
    total_cost = 0
    total_revenue = 0
#Injection Costs
    for rates,dates in zip(injection_rates,injection_dates):
        if current_stored == max_volume:
                rates=0 
        if rates > max_volume:
            rates = max_volume
        if rates + current_stored > max_volume:
             actual_injected = max_volume- current_stored
        else:
             actual_injected = rates
        current_stored += actual_injected
        inject_price_per_unit = gas.at[dates,'LSTM Pred']
        total_cost += inject_price_per_unit * actual_injected
#Withdrawal Costs/Profit
    for rates,dates in zip(withdrawal_rates,withdrawal_dates):
        withdrawal_price_per_unit = gas.loc[dates,'LSTM Pred']
        actual_withdraw = min(rates,current_stored)
        total_revenue +=  actual_withdraw * withdrawal_price_per_unit
        current_stored -= actual_withdraw
#Storage Cost: assuming the cost of storage is charged by months of storing
    for with_dates,inject_dates in zip(withdrawal_dates,injection_dates): 
        time_held = (dater(with_dates) - dater(inject_dates)).days
        print(time_held) # just to check the formating
        per_day_store_cost = storage_costs/31
        total_storage_cost = time_held * per_day_store_cost
        total_cost += total_storage_cost
    profit = total_revenue - total_cost
    print(f'Current Stored: {current_stored}')
    print(f'Total Cost: {total_cost}')
    print(f'Actual Injected(Last Cycle): {actual_injected}')
    print(f'Withdrawed: {actual_withdraw}')
    print(f'Total Storage Cost: {total_storage_cost}')
    print(f'Total Revenue:  {total_revenue}')
    return print(f"Contract Value: {profit}")

price_gas_contract(injection_dates=['2024-10-31','2024-11-30'],withdrawal_dates=['2025-01-31'],injection_rates=[20000,12],withdrawal_rates=[2000000],max_volume=20000,storage_costs=100)

92
Current Stored: 0
Total Cost: 240576.93843533917
Actual Injected(Last Cycle): 0
Withdrawed: 20000
Total Storage Cost: 296.7741935483871
Total Revenue:  251000.2460479737
Contract Value: 10423.307612634526


### Testing Functionality 

In [149]:
print('Example shows handling the case when injection goes over the max volume')
price_gas_contract(injection_dates=['2024-10-31','2024-11-30'],withdrawal_dates=['2025-01-31'],injection_rates=[20000],withdrawal_rates=[20000],max_volume=20000,storage_costs=100)
print('------------------------------------------------')
price_gas_contract(injection_dates=['2024-10-31','2024-11-30'],withdrawal_dates=['2025-01-31'],injection_rates=[20000,20000],withdrawal_rates=[20000],max_volume=20000,storage_costs=100)
print('------------------------------------------------')

print('Handeling the case when withdraw is more than stored')
price_gas_contract(injection_dates=['2024-10-31','2024-11-30'],withdrawal_dates=['2025-01-31'],injection_rates=[100,100],withdrawal_rates=[10000],max_volume=20000,storage_costs=100)
print('------------------------------------------------')

print('Shows storage management,and withdrawal system')
price_gas_contract(injection_dates=['2024-10-31','2024-11-30'],withdrawal_dates=['2025-01-31','2025-01-31'],injection_rates=[1000,500],withdrawal_rates=[1000,400],max_volume=20000,storage_costs=100)
print('------------------------------------------------')

print('Dynamic Selling with dates')
price_gas_contract(injection_dates=['2024-10-31','2024-11-30'],withdrawal_dates=['2025-02-28','2025-01-31'],injection_rates=[1000,500],withdrawal_rates=[1000,500],max_volume=20000,storage_costs=100)
print('------------------------------------------------')
price_gas_contract(injection_dates=['2024-10-31','2024-11-30','2025-07-31'],withdrawal_dates=['2025-01-31','2025-01-31','2025-09-30'],injection_rates=[1000,1000,10000],withdrawal_rates=[1000,1000,10000],max_volume=20000,storage_costs=100)


Example shows handling the case when injection goes over the max volume
92
Current Stored: 0
Total Cost: 240576.93843533917
Actual Injected(Last Cycle): 20000
Withdrawed: 20000
Total Storage Cost: 296.7741935483871
Total Revenue:  251000.2460479737
Contract Value: 10423.307612634526
------------------------------------------------
92
Current Stored: 0
Total Cost: 240576.93843533917
Actual Injected(Last Cycle): 0
Withdrawed: 20000
Total Storage Cost: 296.7741935483871
Total Revenue:  251000.2460479737
Contract Value: 10423.307612634526
------------------------------------------------
Handeling the case when withdraw is more than stored
92
Current Stored: 0
Total Cost: 2718.9356665841997
Actual Injected(Last Cycle): 100
Withdrawed: 200
Total Storage Cost: 296.7741935483871
Total Revenue:  2510.002460479737
Contract Value: -208.9332061044629
------------------------------------------------
Shows storage management,and withdrawal system
92
62
Current Stored: 100
Total Cost: 18614.585664772